In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Filename format: randomTrackGraph_acce_5_.csv
str_random_list = "dataset\\RQ3_dataset\\random\\"
# Complexity of pages.
str_complexity = "dataset\\RQ3_dataset\\list.csv"
# Selection result saved in result_file_dire in RQ3_experiment.
str_result = "GPT_4o_photo.csv"
# result of hit ratio.
str_hit_filename = "RQ3_table_GPT4o_photo_hit.csv"
# result of variability
str_stab_filename = "RQ3_table_GPT4o_photo_stab.csv"

In [3]:
'''
The first index represents the label.
    0: normal
    1-2: minal dismiss, 1 means have the problem(A), 2 means have not the problem(B).
    3-4: medium dismiss, 3A 4B.
    5-6: severe dismiss, 5A 6B.
    7-8: duplicate, 7A 8B.
    9-10: inconsistent, 9A 10B.
    11: minal modify.
    12: large modify.
The second index represents the length, in another way, the complexity.
    0: low complexity
    1: medium
    2: high
    3: on average
'''
np_stat_hit = np.zeros((13,4))
np_stat_stab = np.zeros((13,4))
np_stat_count = np.zeros((13,4))

# np_result: index 0 is filename, format datasetBlueMoon_4_0_0.txt.
# index 1 to 5 is the five results.
np_result = np.array(pd.read_csv(str_result, encoding = "UTF-8"))
for i in range(0, len(np_result)):
    str_dataset = np_result[i][0].replace("dataset", "random").replace(".txt", "")
    # Cutting string into four elements like "randomXXX", "4", "0", "1".
    dataset_list = str_dataset.split("_")
    # index_position means which position this test is, which is the index 1 in np_random_source.
    # index_position is also the answer of this test.
    index_position = int(dataset_list[-1])
    # index_group means which group this test is, which is the index 0 in np_random_source.
    index_group = int(dataset_list[-2])
    # Making the file name of random source table.
    str_random_source = dataset_list[0]
    ad_list = 1
    while len(dataset_list[ad_list]) > 2:
        str_random_source = str_random_source + "_" + dataset_list[ad_list]
        ad_list = ad_list + 1
    str_random_source = str_random_source + "_" + dataset_list[ad_list] + "_.csv"
    np_random_source = np.array(pd.read_csv(str_random_list + str_random_source, encoding = "UTF-8", index_col = 0))
    test_length = len(np_random_source)

    # Check the index in np_stat, which is label and complexity.
    label = 0
    complexity = 0
    # Check complexity.
    np_list_source = np.array(pd.read_csv(str_complexity, encoding = "UTF-8"))
    
     # Making the file name of acitivity list table.
    str_check_list = dataset_list[0].replace("random", "activity")
    ad_list = 1
    while len(dataset_list[ad_list]) > 2:
        str_check_list = str_check_list + "_" + dataset_list[ad_list]
        ad_list = ad_list + 1
    str_check_list = str_check_list + "_" + dataset_list[ad_list] + "_.txt"
    
    for j in range(0, len(np_list_source)):
        if str_check_list == np_list_source[j][0]:
            complexity = int(np_list_source[j][1]) - 1
            break
    # Check label.
    if index_group == 0:
        label = 0
    elif index_group >= 1 and index_group <= 5:
        if np_random_source[index_position][index_group - 1] != 0:
            label = 1
        else:
            label = 2
    elif index_group >= 6 and index_group <= 10:
        if np_random_source[index_position][index_group - 1] != 0:
            label = 3
        else:
            label = 4
    elif index_group >= 12 and index_group <= 15:
        if np_random_source[index_position][index_group - 1] != 0:
            label = 5
        else:
            label = 6
    elif index_group >= 16 and index_group <= 20:
        if np_random_source[index_position][index_group - 1] != 0:
            label = 7
        else:
            label = 8
    elif index_group >= 21 and index_group <= 25:
        if np_random_source[index_position][index_group - 1] != 0:
            label = 9
        else:
            label = 10
    elif index_group == 26:
        label = 11
    elif index_group == 27:
        label = 12
    else:
        continue

    # initialize two parameters.
    hit_ratio = 0.0
    stability = 0.0

    # Calculate numbers appear most.
    appear_list = np.zeros(test_length + 1)
    for j in range(1, 6):
        if np_result[i][j] > test_length:
            np_result[i][j] = 0
        appear_list[int(np_result[i][j])] = appear_list[int(np_result[i][j])] + 1
    appear_result = 0
    appear_time = 0
    for j in range(1, len(appear_list + 1)):
        if appear_list[j] > appear_time:
            appear_time = appear_list[j]
            appear_result = j
            
    # Calculate the stability.
    for j in range(1, 6):
        if np_result[i][j] == index_position + 1:
            hit_ratio = hit_ratio + 1
        if np_result[i][j] == 0:
            stability = stability + test_length ** 2
        else:
            stability = stability + (np_result[i][j] - appear_result) ** 2
    hit_ratio = hit_ratio / 5
    stability = ((stability / 5) ** 0.5) / test_length

    # Write the result to its complexity and average.
    np_stat_hit[label][complexity] = np_stat_hit[label][complexity] + hit_ratio
    np_stat_stab[label][complexity] = np_stat_stab[label][complexity] + stability
    np_stat_count[label][complexity] = np_stat_count[label][complexity] + 1

    np_stat_hit[label][3] = np_stat_hit[label][3] + hit_ratio
    np_stat_stab[label][3] = np_stat_stab[label][3] + stability
    np_stat_count[label][3] = np_stat_count[label][3] + 1

for j in range(0, np_stat_count.shape[1]):
    np_stat_hit[2][j] = np_stat_hit[2][j] + np_stat_hit[1][j]
    np_stat_stab[2][j] = np_stat_stab[2][j] + np_stat_stab[1][j]
    np_stat_count[2][j] = np_stat_count[2][j] + np_stat_count[1][j]

    np_stat_hit[4][j] = np_stat_hit[4][j] + np_stat_hit[3][j]
    np_stat_stab[4][j] = np_stat_stab[4][j] + np_stat_stab[3][j]
    np_stat_count[4][j] = np_stat_count[4][j] + np_stat_count[3][j]

    np_stat_hit[6][j] = np_stat_hit[6][j] + np_stat_hit[5][j]
    np_stat_stab[6][j] = np_stat_stab[6][j] + np_stat_stab[5][j]
    np_stat_count[6][j] = np_stat_count[6][j] + np_stat_count[5][j]

    np_stat_hit[8][j] = np_stat_hit[8][j] + np_stat_hit[7][j]
    np_stat_stab[8][j] = np_stat_stab[8][j] + np_stat_stab[7][j]
    np_stat_count[8][j] = np_stat_count[8][j] + np_stat_count[7][j]

    np_stat_hit[10][j] = np_stat_hit[10][j] + np_stat_hit[9][j]
    np_stat_stab[10][j] = np_stat_stab[10][j] + np_stat_stab[9][j]
    np_stat_count[10][j] = np_stat_count[10][j] + np_stat_count[9][j]

for i in range(0, np_stat_count.shape[0]):
    for j in range(0, np_stat_count.shape[1]):
        np_stat_hit[i][j] = np_stat_hit[i][j] / np_stat_count[i][j]
        np_stat_stab[i][j] = np_stat_stab[i][j] / np_stat_count[i][j]

with open(str_hit_filename, "w", encoding = "UTF-8") as f:
    f.write("label,1,2,3,avg")
    for i in range(0, np_stat_count.shape[0]):
        f.write("\n" + str(i))
        for j in range(0, np_stat_count.shape[1]):
            f.write("," + str(np_stat_hit[i][j]))

with open(str_stab_filename, "w", encoding = "UTF-8") as f:
    f.write("label,1,2,3,avg")
    for i in range(0, np_stat_count.shape[0]):
        f.write("\n" + str(i))
        for j in range(0, np_stat_count.shape[1]):
            f.write("," + str(np_stat_stab[i][j]))